In [44]:
corpus = sc.textFile("WikipediaPagesOneDocPerLine1000LinesSmall.txt")

In [45]:
numberOfDocs = corpus.count()

In [46]:
validLines = corpus.filter(lambda x : 'id' in x and 'url=' in x)

In [47]:
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))

In [48]:
import re
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

In [49]:
allWords =keyAndListOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))

In [50]:
from operator import add
allCounts = allWords.reduceByKey(add)

In [51]:
topWords = allCounts.top(20000, key=lambda x: x[1])

In [52]:
twentyK = sc.parallelize(range(20000))

In [53]:
dictionary = twentyK.map (lambda x : (topWords[x][0], x))

In [54]:
allWords = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

In [55]:
allDictionaryWords = dictionary.join(allWords)

In [56]:
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))

In [57]:
import numpy as np

In [58]:
def stringVector (x):
    returnVal= str(x[0])
    for j in x[1]:
        returnVal += ','+ str(j)
    return returnVal

In [59]:
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

In [60]:
forCSV= allDictionaryWordsInEachDoc.mapValues(lambda x: np.array(list(x)))

In [61]:
print(forCSV.take(5))

[('431971', array([    1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     7,     7,
           7,     7,     7,     7,     7,     7,     7,     7,     7,
          11,    11,    11,    36,    38,    40,    40,    40,    50,
          68,    68,    95,    95,    95,   117,   117,   117,   157,
         162,   162,   162,   278,   278,   278,   393,   518,   538,
         554,   619,   904,  1000,  1074,  1074,  1074,  1074,  1074,
        1211,  1397,  1418,  1418,  1418,  1418,  1418,  1784,  1788,
        1788,  1967,  1967,  1992,  1992,  2225,  2267,  2267,  2267,
        2267,  2267,  2267,  2267,  2296,  2594,  2594,  2594,  2797,
        3435,  3734,  3735,  4366,  4671,  4828,  8148, 12502, 16567,
           2,     2,     2,     2,     2,     2,     2,     2,     2,
           2,     2,     2,     2,     2,     2,     2,     2,     2,
           2,     2,     2,     2,     2,     2,     2,     2,     2,
        

In [62]:
def buildArray (listOfIndices):
    returnVal = np.zeros (20000)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    mysum = np.sum (returnVal)
    returnVal = np.divide (returnVal, mysum)
    return returnVal

In [63]:
tfs = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))

In [64]:
tfs.take(5)

[('431971',
  array([0.08553655, 0.02488336, 0.05132193, ..., 0.        , 0.        ,
         0.        ])),
 ('431999',
  array([0.06479482, 0.03239741, 0.03887689, ..., 0.        , 0.        ,
         0.        ])),
 ('432000',
  array([0.07192043, 0.04475899, 0.02371844, ..., 0.        , 0.        ,
         0.        ])),
 ('432001',
  array([0.08540752, 0.03318692, 0.02781845, ..., 0.        , 0.        ,
         0.        ])),
 ('432035',
  array([0.07300672, 0.04130644, 0.05091258, ..., 0.        , 0.        ,
         0.        ]))]

In [65]:
def zeroOneArray(listOfTFs):
    returnVal = np.zeros(20000)
    for i in range(len(listOfTFs)):
        if listOfTFs[i] != 0.0:
            returnVal[i] = 1.0
    return returnVal

In [66]:
zeroOrOne = tfs.map(lambda x: (x[0], zeroOneArray(x[1])))

In [67]:
zeroOrOne.take(5)

[('431971', array([1., 1., 1., ..., 0., 0., 0.])),
 ('431999', array([1., 1., 1., ..., 0., 0., 0.])),
 ('432000', array([1., 1., 1., ..., 0., 0., 0.])),
 ('432001', array([1., 1., 1., ..., 0., 0., 0.])),
 ('432035', array([1., 1., 1., ..., 0., 0., 0.]))]

In [68]:
#dfs = zeroOrOne.map(lambda x: (x[0], x[1].sum()))
dfs = zeroOrOne.reduce (lambda x1, x2:(("", np.add (x1[1], x2[1]))))[1]

In [69]:
dfs

array([919., 920., 869., ...,   1.,   1.,   1.])

In [70]:
multiplier = np.full (20000, numberOfDocs)
idfs = np.log (np.divide(numberOfDocs, dfs))

In [71]:
allDocsAsNumpyArraysTFidf = tfs.map (lambda x: (x[0], np.multiply (x[1], idfs)))

In [73]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
allDocsAsNumpyArraysTFidf.take(1)

[('431971', array([0.0072252 , 0.00207481, 0.00720622, 0.00384264, 0.00075779,
         0.00249281, 0.00894812, 0.00436052, 0.00217599, 0.00753549,
         0.00035093, 0.00156519, 0.00978827, 0.00288784, 0.0005816 ,
         0.        , 0.00046408, 0.00481715, 0.00062748, 0.00320754,
         0.        , 0.00750615, 0.        , 0.        , 0.        ,
         0.00092412, 0.        , 0.        , 0.0006965 , 0.        ,
         0.00238332, 0.00192236, 0.00104111, 0.00225072, 0.        ,
         0.00182019, 0.00124531, 0.00215598, 0.00096118, 0.00243568,
         0.00297092, 0.00193972, 0.        , 0.00119756, 0.        ,
         0.00124185, 0.        , 0.        , 0.        , 0.00132344,
         0.00138662, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.00142892, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00182646, 0.        , 0.00279609,
         0.00344841, 0.        , 0.00153789, 0.00293678, 0.        ,
         0.        , 0. 

In [36]:
idfs = dfs.map(lambda x: (x[0] ,np.log(numberOfDocs / x[1]))) 

AttributeError: 'numpy.ndarray' object has no attribute 'map'

In [28]:
idfs.take(5)

[('431971', 2.05572501506252),
 ('431999', 2.137070654516472),
 ('432000', 1.0671136216087387),
 ('432001', 1.2447947988461912),
 ('432035', 1.5896352851379207)]

In [29]:
tfidfs = tfs.join(idfs).map(lambda x: (x[0], x[1][0] * x[1][1] ))

In [30]:
tfidfs.take(5)

[('431971', array([0.25813899, 0.07509498, 0.15488339, 0.09856216, 0.02816062,
         0.06101467, 0.09386872, 0.0516278 , 0.02816062, 0.07978841,
         0.00469344, 0.01408031, 0.12202934, 0.03285405, 0.00469344,
         0.        , 0.00469344, 0.01877374, 0.00469344, 0.02346718,
         0.        , 0.03754749, 0.        , 0.        , 0.        ,
         0.00469344, 0.        , 0.        , 0.00469344, 0.        ,
         0.01408031, 0.00938687, 0.00469344, 0.00938687, 0.        ,
         0.00938687, 0.00469344, 0.00938687, 0.00469344, 0.00938687,
         0.01408031, 0.00938687, 0.        , 0.00469344, 0.        ,
         0.00469344, 0.        , 0.        , 0.        , 0.00469344,
         0.00469344, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.00469344, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00469344, 0.        , 0.00938687,
         0.00938687, 0.        , 0.00469344, 0.00938687, 0.        ,
         0.        , 0. 

In [ ]:
def getPrediction(textInput, k):
    # make rdd for document 
    target = sc.parallelize(('', textInput))
    # (word, 1)
    wordsArray = target.flatmap(lambda x: (i, 1) for i in regex.sub(' ', x).split())
    # (doc (dicPosition, 1))
    dictionaryWords = dictionary.join(wordsArray)
    # calculate cosine similality between textnput and all docs
    # select top k similarity (docID, similaliry)
    # count how many time each docID appear in the top k similar docID
    # take top 1